In [2]:
#from google.colab import drive
# drive.mount('/content/drive')

# Importación de librerías

In [3]:
# Librerías estándar
import os
import sys
import warnings

# Manipulación de datos
import pandas as pd
import numpy as np

# Configuración de warnings
warnings.filterwarnings('ignore')

# Análisis de nulos
import missingno as msno

# Visualización de datos
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Textos
import unicodedata
from fuzzywuzzy import process
import re

In [4]:
#import sys

#sys.path.append('/content/drive/MyDrive/00_Nuclio/Ediciones/TFM_fintech/02_scripts/')

#from funciones import *

# Ruta archivos



In [5]:
# Definir ruta donde están los ficheros
path = r'C:\Ricardo\Cursos\Nuclio Digital School\Master Dual Data & AI\Entregables\TFM Fintech'

# Funciones

In [6]:
# Funcion carga de archivos.
def carga_archivos(path):
    # Lista de los archivos en el directorio.
    lista = os.listdir(path)
    print(lista)

    # Filtrado archivos .csv y .xlsx.
    files = [file for file in lista if file.endswith('.csv') | file.endswith('.xlsx')]

    print('*' * 80)
    return files
      
files=carga_archivos(path)

FileNotFoundError: [WinError 3] El sistema no puede encontrar la ruta especificada: 'C:\\Ricardo\\Cursos\\Nuclio Digital School\\Master Dual Data & AI\\Entregables\\TFM Fintech'

In [ ]:
# Funcion para leer archivos CSV o Excel.
def leer_archivos(ruta_completa):
    try:
        ruta_completa=ruta_completa.strip()
        _, extension = os.path.splitext(ruta_completa.lower())

        if extension == '.csv':
            df = pd.read_csv(ruta_completa, sep=None,engine='python') # Forzamos que python interprete el separador del .csv
        elif extension in ('.xlsx', '.xls'):
            df = pd.read_excel(ruta_completa)
        else:
            print("Error: Formato no compatible")
            return None

        return df

    except FileNotFoundError:
        print(f"Error: Archivo no encontrado en la ruta '{ruta_completa}'.")
        return None

    except Exception as e:
        print(f"Error inesperado: {e}")
        return None


In [ ]:
# Funcion exploracion inicial de datos.
def exploracion_datos(df):
    print('Exploración inicial de datos:')
    print('*'*100)

    # Información general del dataframe.
    num_filas, num_columnas = df.shape
    print(f'El numero de filas es: {num_filas}\nEl numero de columnas es: {num_columnas}')
    print('*'*100)

    # Exploracion visulal de las primeras, últimas y aleatorias filas del dataframe.
    print('Las 5 primeras filas del dataframe son:')
    display(df.head())
    print('*'*100)
    print('Las 5 últimas filas del dataframe son:')
    display(df.tail())
    print('*'*100)
    print('Muestra aleatoria de 5 filas del dataframe:')
    display(df.sample(5))
    print('*'*100)

    # Estadisticos descriptivos del dataframe.
    print('Estadísticos descriptivos del dataframe:')
    display(df.describe())
    print('*'*100)

    # Resumen de tipologia de datos, visualizacion de nulos y valores unicos.
    print('Resumen de tipología de datos, visualización de nulos y valores únicos:')
    df_tipos=df.dtypes.to_frame(name='Tipos de datos')
    df_nulos=df.isnull().sum().to_frame(name='Nulos')
    df_porc_nulos = (df.isnull().sum() / len(df) * 100).to_frame(name='Porcentaje Nulos')
    df_valores_unicos = pd.DataFrame(df.apply(lambda x: x.unique()))
    df_valores_nunicos = pd.DataFrame(df.apply(lambda x: x.nunique()))
    df_por_valores_nunicos=pd.DataFrame(df.apply(lambda x: x.nunique())/df.shape[0]*100)
    df_valores_unicos.rename(columns={0:'Valores unicos'}, inplace=True)
    df_valores_nunicos.rename(columns={0:'Numero valores unicos'}, inplace=True)
    df_por_valores_nunicos.rename(columns={0:'Porcentaje valores unicos'}, inplace=True)
    df_exploracion = pd.concat([df_tipos, df_nulos, df_porc_nulos,df_valores_nunicos,df_por_valores_nunicos,df_valores_unicos], axis=1)
    
    # MOSTRAR el resumen final
    display(df_exploracion)
    print('*'*100)
    
    return df_exploracion

In [ ]:
# Funcion para renombrar columnas originales.
def renombrar_columnas(df):
    nombre_columnas = ['age', 'job', 'marital_status', 'education', 'credit_default', 'housing_loan', 'personal_loan',
       'contact_type', 'last_contact_month', 'last_contact_day', 'last_contact_duration_secs', 
       'number_contacts', 'number_days_last_contact','numbrer_of_previous_contacts', 
       'outcome_previous_campaign', 'employement_vaiation_rate', 'consumer_price_index','consumer_confidence_index', 
       'euribor_3m', 'number_employees', 'subscribed_term_deposit']
    df.columns = nombre_columnas
    
    return df

In [ ]:
# Funcion para normalizar textos.
def normalizar_textos(df):
    for col in df.select_dtypes(include=['object']).columns: # Iteramos sobre columnas de tipo object.
        df[col]=df[col].str.lower() # Convertimos a minúsculas.
        df[col]=df[col].str.strip() # Eliminamos espacios en blanco al inicio y final.
        df[col]=df[col].str.replace(r'\.+$', '', regex=True) # Eliminamos los '.' al final de las cadenas de texto.
        df[col]=df[col].str.replace(r'(?<=\w)\.+(?=\w)', '_', regex=True) # Reemplazamos los '.' entre palabras por '_'.
        df[col]=df[col].str.replace(r'(?<=\w)-+(?=\w)', '_', regex=True) # Reemplazamos los '-' entre palabras por '_'.
        dict_job={'admin':'administrative_staff'}
        if col=='job':
            df[col]=df[col].replace(dict_job)
            
    return df

In [ ]:
# Funcion convertir columnas a tipo categórico.
def columnas_categoricas(df):
    # Listas categorias.
    lista_categorias_job=['unknown','unemployed','student','retired','housemaid', 'services','blue_collar',
                        'self_employed',  'administrative_staff',  'technician','entrepreneur','management']

    lista_marital_status=['unknown','single','married','divorced']

    lista_categorias_education=['unknown','illiterate','basic_4y','basic_6y', 'basic_9y', 
                                'high_school', 'professional_course',  'university_degree']

    lista_credit_default=['unknown','yes','no']

    lista_housing_loan=['unknown','yes','no']

    lista_personal_loan=['unknown','yes','no']

    lista_contact_type=['telephone','cellular']

    lista_last_contact_month=['mar', 'apr','may', 'jun', 'jul', 'aug','sep', 'oct', 'nov', 'dec']

    lista_last_contact_day=['mon', 'tue', 'wed', 'thu', 'fri']

    lista_outcome_previous_campaign=['nonexistent', 'failure', 'success']

    # Convertir columnas a tipo categórico segun las listas definidas.
    df['job'] = pd.Categorical(df['job'], categories=lista_categorias_job, ordered=False)
    df['marital_status'] = pd.Categorical(df['marital_status'], categories=lista_marital_status, ordered=False)
    df['education'] = pd.Categorical(df['education'], categories=lista_categorias_education, ordered=False)
    df['credit_default'] = pd.Categorical(df['credit_default'], categories=lista_credit_default, ordered=False)
    df['housing_loan'] = pd.Categorical(df['housing_loan'], categories=lista_housing_loan, ordered=False)
    df['personal_loan'] = pd.Categorical(df['personal_loan'], categories=lista_personal_loan, ordered=False)
    df['contact_type'] = pd.Categorical(df['contact_type'], categories=lista_contact_type, ordered=False)
    df['last_contact_month'] = pd.Categorical(df['last_contact_month'], categories=lista_last_contact_month, ordered=True)
    df['last_contact_day'] = pd.Categorical(df['last_contact_day'], categories=lista_last_contact_day, ordered=False)
    df['outcome_previous_campaign'] = pd.Categorical(df['outcome_previous_campaign'], categories=lista_outcome_previous_campaign, ordered=False)
    
    return df

In [ ]:
# Funcion cambiar columna subscribed_term_deposit a binario.
def normalizar_binario(df):
    dict_subscribed_term_deposit={'yes':1,'no':0} #Creamos un dicionario.
    nulos=set(df.subscribed_term_deposit)-set(dict_subscribed_term_deposit.keys()) # Comprobamos si hay valores no contemplados en el diccionario.
    if nulos:
        print(f'Existen valores nulos en la columna subscribed_term_deposit: {nulos}') # Si hay valores no contemplados, los mostramos.
    df.subscribed_term_deposit=df.subscribed_term_deposit.replace(dict_subscribed_term_deposit)
    df.subscribed_term_deposit = df.subscribed_term_deposit.astype('int64') # Convertimos a int64.

    return df

# Carga de datos

In [ ]:
# Pasar la funcion para cargar archivos.
carga_archivos(path)

['01_exploracion.ipynb', '2_marketing_fintech_guía_proyecto.docx', 'bank-additional_bank-additional-full.csv', 'data_info.rtf', 'Fintech', 'Fintech.zip']
********************************************************************************


['bank-additional_bank-additional-full.csv']

In [ ]:
# Creamos un dicionario vacio para añadir archivos.
dict_data={}

for x in files:
    ruta=os.path.join(path, x)
    df_temp=leer_archivos(ruta)
    dict_data[x]=df_temp
    print(f"{x}: {df_temp.shape}") 

# Limpiar variable temporal
del df_temp

bank-additional_bank-additional-full.csv: (41188, 21)


In [ ]:
dict_data.keys()

dict_keys(['bank-additional_bank-additional-full.csv'])

# Exploración inicial de los datos



In [ ]:
# Creamos un dicionario vacio para la exploracion
dict_exploracion = {}

for k, v in dict_data.items():
    print(f"EXPLORANDO: {k}")
    print("="*120)
    
    # Ejecutamos y guardamos la exploracion.
    exploracion=exploracion_datos(v)
    dict_exploracion[k]=exploracion
    
    print("\n" + "="*120 + "\n")

NameError: name 'dict_data' is not defined

# Limpieza de datos

In [ ]:
# Asignamos a la variable df_bank el archivo bank-additional_bank-additional-full.csv
df_bank=dict_data['bank-additional_bank-additional-full.csv']

In [ ]:
# Ejecutamos las funciones de renombrar columnas, normalizar textos y crear las columnas categoricas.
renombrar_columnas(df_bank)
normalizar_textos(df_bank)
columnas_categoricas(df_bank)

,age,job,marital_status,education,credit_default,housing_loan,personal_loan,contact_type,last_contact_month,last_contact_day,...,number_contacts,number_days_last_contact,numbrer_of_previous_contacts,outcome_previous_campaign,employement_vaiation_rate,consumer_price_index,consumer_confidence_index,euribor_3m,number_employees,subscribed_term_deposit
0,56,housemaid,married,basic_4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high_school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high_school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,administrative_staff,married,basic_6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high_school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional_course,no,yes,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes
41184,46,blue_collar,married,professional_course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41185,56,retired,married,university_degree,no,yes,no,cellular,nov,fri,...,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41186,44,technician,married,professional_course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes


In [ ]:
# Creamos una copia para pasar la funcion normalizar_binario de la columna subscribed_term_deposit.
df_binario=df_bank.copy()
normalizar_binario(df_binario)

,age,job,marital_status,education,credit_default,housing_loan,personal_loan,contact_type,last_contact_month,last_contact_day,...,number_contacts,number_days_last_contact,numbrer_of_previous_contacts,outcome_previous_campaign,employement_vaiation_rate,consumer_price_index,consumer_confidence_index,euribor_3m,number_employees,subscribed_term_deposit
0,56,housemaid,married,basic_4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
1,57,services,married,high_school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
2,37,services,married,high_school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
3,40,administrative_staff,married,basic_6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
4,56,services,married,high_school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional_course,no,yes,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,1
41184,46,blue_collar,married,professional_course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,0
41185,56,retired,married,university_degree,no,yes,no,cellular,nov,fri,...,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,0
41186,44,technician,married,professional_course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,1


In [ ]:
df_binario.dtypes

age                                int64
job                             category
marital_status                  category
education                       category
credit_default                  category
housing_loan                    category
personal_loan                   category
contact_type                    category
last_contact_month              category
last_contact_day                category
last_contact_duration_secs         int64
number_contacts                    int64
number_days_last_contact           int64
numbrer_of_previous_contacts       int64
outcome_previous_campaign       category
employement_vaiation_rate        float64
consumer_price_index             float64
consumer_confidence_index        float64
euribor_3m                       float64
number_employees                 float64
subscribed_term_deposit            int64
dtype: object